# Sektion 3 - Time-dependent routing

## Functions

In [ ]:
def get_unique_nodes(edge_list):
    """
    Assembles a list of all unique nodes appearing in a given graph.
    The start node is added by default; every other node is only included
    if it can be reached from another node. Isolated nodes will not be 
    added to the list, unless they happen to be the start node.
    """

    nr_nodes = 0
    unique_nodes = []
    unique_nodes.append(0)

    for edge in edge_list:
        if edge[1] not in unique_nodes:
            unique_nodes.append(edge[1])

    return unique_nodes


In [ ]:
def construct_sliced_graph(edge_list, start, dest):
    """
    Constructs a two-dimensional array from a list of edges. Each subarray represents 
    one time slice. The returned array contains the specified start node in the 
    first sub-array; based on this start node, every node that can be reached in 
    n steps is present in the nth subarray. Nodes that can be reached in a different 
    amount of steps are present in every fitting subarray.
    """
    sg = [] # sliced graph
    sg.append([start])
    node_counter = 1


    total_nodes = len(get_unique_nodes(edge_list))

    for k in range(total_nodes):
    
        if len(sg[k]) != 0 and not (len(sg[k]) == 1 and sg[k][0] == dest):
            sg.append([])
            for j in range(len(edge_list)):
                # for a given edge j in the format a->b:
                # if a is in previous slice and b is not in this slice, and if b is not the start node, add it to the slice
                if edge_list[j][0] in sg[k] and edge_list[j][1] not in sg[k+1] and edge_list[j][1] != start:
                    sg[k+1].append(edge_list[j][1])
        else:
            break

    # necessary if graph is not acyclic
    if dest in sg[-1]:
        sg[-1] = [dest]
    else:
        return None 
        # TODO - error handling

    # the last node is always the destination node.
    # therefore we can eliminate all nodes in the next-to-last slice that do not lead to the destination node.
    # going back that way, we can eliminate nodes that won't lead to our destination.
    
    nr_slices_to_remove = 0

    for i in range(len(sg) - 1):
        nodes_to_remove = []
        index = len(sg) - 1 - i
        for j in range(len(sg[index - 1])):
            leads_to_next_slice = 0 # number of edges in the next slice that sg[index-1][j] is connected to
            for k in range(len(sg[index])):

                # for each node in the next slice, add 1 if sg[index-1][j] is connected to it, 0 if not
                leads_to_next_slice += next((1 for u, v in enumerate(edge_list) if v[0] == sg[index-1][j] and v[1] == sg[index][k]), 0) 

            # if leads_to_next_slice is zero, it means sg[index-1][j] is not connected to any node in the next slice; essentially a dead end. 
            # it is removed, and any node in the previous slice that only connected to it will thus also be a dead end, and will be removed. etc., etc.
            if leads_to_next_slice == 0:
                nodes_to_remove.append(sg[index-1][j])

        for node in nodes_to_remove:
            sg[index-1].remove(node)

    slices_to_remove = []
    for i in range(len(sg)):
        if len(sg[len(sg) - 1 - i]) == 1 and dest in sg[len(sg) - 1 - i]:
            slices_to_remove.append(len(sg) - 1 - i)
        else:
            break
    
    if len(slices_to_remove) > 1:
        for i in slices_to_remove[:-1]:
            del sg[i]

    return sg

In [ ]:
def construct_numbered_sliced_graph(sg): 
    nsg = [] # numbered sliced graph
    node_counter = 0

    for m in range(len(sg)):
        nsg.append([])
        for n in range(len(sg[m])):
            nsg[m].append(node_counter)
            node_counter += 1
    
    return nsg

In [ ]:
def construct_graph_matrix(edge_list):
    """
    Constructs a dataframe that contains all edges present in 
    the graph. The returned dataframe can contain several edges 
    per two nodes. The entry at matrix[0][1] contains an array 
    of all edges that go from node 0 to node 1.
    """

    node_labels = get_unique_nodes(edge_list)
    graph_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)
    
    for edge in edge_list:
        start = edge[0]
        end = edge[1]
        graph_matrix[start][end].append(edge)
    
    return graph_matrix

In [ ]:
def construct_cost_matrix(edge_matrix):
    """
    Constructs a dataframe that contains the cost for the edge 
    between any given pair of nodes a and b. If there is no 
    outgoing edge from a to b, the cost will be equal to the
    fixed penalty value. If there is exactly one edge, its cost
    will be used. If there are several, the smallest cost is 
    used.
    """

    node_labels = edge_matrix.columns
    cost_matrix = pd.DataFrame([[[] for _ in range(len(node_labels))] for _ in range(len(node_labels))], node_labels, node_labels)

    for i in node_labels:
        for j in node_labels:
            entries = edge_matrix[i][j]
            if len(entries) == 0:
                cost_matrix[i][j] = (penalty_value)
            elif len(entries) == 1:
                cost_matrix[i][j] = (entries[0][2]) ### TODO -- anpassen
            else:
                best_cost = penalty_value # penalty by default bigger than the biggest edge cost
                for entry in entries:
                    if entry[2] < best_cost:
                        best_cost = entry[2]
                cost_matrix[i][j] = best_cost
    return cost_matrix

In [ ]:
def find_index(node, time_slice, sliced_graph):
    """
    For a given node, time slice, and sliced graph, determines the qubit index 
    of the node. If the node is not present in the given slice in the graph, 
    returns -1.
    """
    
    numbered_graph = construct_numbered_sliced_graph(sliced_graph)
    for i in range(len(sliced_graph[time_slice])):
        if sliced_graph[time_slice][i] == node:
            return numbered_graph[time_slice][i]
    else:
        return -1


In [ ]:
def fix_nodes(start, destination, sliced_graph, cost_func):
    """
    In the given cost function, fixes specific nodes according to already known 
    values for the qubits. The qubits representing the start and destination nodes 
    are fixed at 1; any other nodes in the last time slice are fixed at 0.
    """
    
    # set start node to 1
    start_node_index = 'X_' + str(find_index(start, 0, sliced_graph))
    cost_func.linear_constraint(linear={start_node_index: 1}, sense='==', rhs=1, name='start_node')

    # set destination node to 1
    dest_node_index = 'X_' + str(find_index(destination, -1, sliced_graph))
    cost_func.linear_constraint(linear={dest_node_index: 1}, sense='==', rhs=1, name='destination_node')

    # set any other nodes in the last slice to 0 # TODO -- is this section necessary?
    #counter = 0
    #for i in sliced_graph[-1]:
    #    if i != destination:
    #        index = 'X_' + str(find_index(i, -1, sliced_graph))
    #        cost_func.linear_constraint(linear={index: 1}, sense='==', rhs=0, name=('last_slice_constraint_' + str(counter)))

    # only the start node is in the first slice. so no iteration over other nodes needed

    return

In [ ]:
def get_node_info_from_index(qubit_index, sg, nsg):
    """ For a given qubit index, returns the corresponding time slice and the node label. """

    for time_slice in range(len(nsg)):
        if qubit_index in nsg[time_slice]:

            pos = nsg[time_slice].index(qubit_index)
            node_label = sg[time_slice][pos]

            return time_slice, node_label

In [ ]:
def result_evaluation(result, sg, nsg, cost_matrix):
    """
    For a given qubit string, returns an explanation of the nodes that are passed per time slice.
    """
    counter = 0
    previous_node = None
    total_cost = 0
    solution_not_valid = False

    for i in range(len(result)):
        if result[i] == '1':
            time_slice, node_label = get_node_info_from_index(i, sg, nsg)
            if time_slice != counter:
                solution_not_valid = True

            if counter == 0:
                previous_node = node_label
            else:
                cost = cost_matrix[previous_node][node_label]
                total_cost += cost
                print(('\t\t\t\tcost: {}').format(cost))
                previous_node = node_label


            print(('node {} at time slice {}').format(node_label, time_slice))

            counter += 1

            
    print(('\ntotal cost: {}').format(total_cost))
    if total_cost >= 500 or solution_not_valid:
        print('\nIt seems like no valid solution could be found. This solution disregards at least one constraint.')

    return

## Preprocessing

In [ ]:
import sympy as sym
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
graph = [(0, 1, 6.0), (0, 2, 4.0), (0, 3, 1.0), (0, 5, 2.0), 
(1, 6, 1.0), 
(2, 4, 3.0), (2, 6, 2.0), (2, 9, 5.0), 
(3, 2, 2.0), (3, 4, 4.0), 
(4, 7, 4.0), (4, 9, 2.0), 
(5, 1, 5.0), 
(6, 8, 3.0), (6, 9, 3.0), 
(7, 9, 3.0),
(9, 8, 4.0)]

#graph = [(0, 1, 5.0), (0, 2, 4.0), (0, 3, 4.0), 
#(1, 4, 3.0), 
#(2, 1, 2.0), (2, 3, 1.0),
#(3, 4, 2.0), (3, 5, 3.0), 
#(4, 6, 2.0),
#(5, 7, 4.0)
#]

start_node = 0
destination_node = 9
penalty_value = 50

In [ ]:
graph = [edge for edge in graph if edge[0] != destination_node and edge[1] != start_node] # discard all edges that go out from destination, and all edges that lead to start
graph.append((destination_node, destination_node, 0.0))
graph

In [ ]:
unique_nodes = get_unique_nodes(graph)
nr_nodes = len(unique_nodes)

In [ ]:
test_sg = construct_sliced_graph(graph, start_node, destination_node)

nr_qubits = 0
for i in range(len(test_sg)):
    for j in range(len(test_sg[i])):
        nr_qubits += 1

nr_qubits

In [ ]:
test_sg

In [ ]:
test_nsg = construct_numbered_sliced_graph(test_sg)
test_nsg

In [ ]:
matrix = construct_graph_matrix(graph)
matrix

In [ ]:
edge_cost_matrix = construct_cost_matrix(matrix)
edge_cost_matrix

In [ ]:
test_nsg = construct_numbered_sliced_graph(test_sg)
test_nsg

## Cost Function

In [ ]:
X = sym.IndexedBase('X')
c = sym.symbols('c')
v = sym.symbols('v')
y = sym.symbols('y')
q = sym.symbols('q')
w = sym.symbols('w')
P = sym.symbols('P')

d = sym.IndexedBase('d')

sg = sym.IndexedBase('sg')
nsg = sym.IndexedBase('nsg')

nrslices = sym.symbols('nrslices')
lenslice = sym.IndexedBase('lenslice')

In [ ]:
cost_function = sym.Sum(                                # iteration over slices
                    (sym.Sum(                           # iteration over nodes in that slice. in each slice, exactly one node needs to be passed
                        X[nsg[c,v]],                    
                        (v, 0, lenslice[c] - 1)
                        )
                        - 1 )**2 * P,

                    (c, 0, nrslices-1)) + 0.5 * sym.Sum(   
                        2 * (sym.Sum(  sym.Sum(  X[nsg[q,y]] * X[nsg[q+1,w]] * d[sg[q,y], sg[q+1,w]],  (w, 0, lenslice[q + 1] - 1) ) , (y, 0, lenslice[q] - 1))),
                        (q, 0, nrslices - 2))

# TODO. an dieser stelle derzeit noch ein workaround mit 0.5 * 2, da aus irgendeinem grund die dreifache summe ohne den zwischenfaktor 2 nicht evaluiert wird??
                    
cost_function

In [ ]:
# translation of data into dictionaries for sympy
single_valued_dict = {
    nrslices: len(test_sg),
    P: penalty_value
    }

numbered_sliced_graph_dict = {
    nsg[i, j]: test_nsg[i][j] for i in range(len(test_nsg)) for j in range(len(test_nsg[i]))
}

sliced_graph_dict = {
    sg[i, j]: test_sg[i][j] for i in range(len(test_sg)) for j in range(len(test_sg[i]))
}

len_slice_dict = {
    lenslice[i]: len(test_sg[i]) for i in range(len(test_sg))
}

d_dict = {
    d[i, j]: edge_cost_matrix[i][j] 
    for i in unique_nodes
    for j in unique_nodes
}

# definition of the cost polynomial
cost_poly = sym.Poly(cost_function
                     .subs(single_valued_dict)
                     .doit()
                     .subs(len_slice_dict)
                     .doit()
                     .subs(numbered_sliced_graph_dict)
                     .subs(sliced_graph_dict)
                     .doit()
                     .subs(d_dict)
                     .doit(),
                     [X[i] for i in range(nr_qubits)])
cost_poly

In [ ]:
edge_cost_matrix[0][2]

## QAOA

In [ ]:
import qiskit
from qiskit.algorithms import QAOA, VQE
import time

from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer, CplexOptimizer, GroverOptimizer
from qiskit.utils import QuantumInstance
from qiskit_optimization.problems import QuadraticProgram
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

# generate qiskit's cost function
qiskit_cost_function = QuadraticProgram()

# define qiskit variables
for i in range(nr_qubits):
    qiskit_cost_function.binary_var('X_' + str(i))

# specify qiskit cost function
qiskit_cost_function.minimize(
    linear = [int(cost_poly.coeff_monomial(X[i]**1)) for i in range(nr_qubits)],
    quadratic = {
        ('X_'+str(i), 'X_'+str(j)): cost_poly.coeff_monomial(X[i]**1 * X[j]**1)
        for i in range(nr_qubits)
        for j in range(i,nr_qubits)
    }
    )
fix_nodes(start_node, destination_node, test_sg, qiskit_cost_function)

print(qiskit_cost_function.export_as_lp_string())

In [ ]:
def start_dest_constraints(vso, vti, cost_func):
    """
    Adds constraints to ensure that there is exactly one outgoing edge from the start 
    and one incoming edge to the target vertex.
    """
    linear_dict_start = {}
    linear_dict_target = {}

    for i in range(len(vso)):
        linear_dict_start.update({'X_' + str(vso[i]): 1})

    for i in range(len(vti)):
        linear_dict_target.update({'X_' + str(vti[i]): 1})

    cost_func.linear_constraint(linear=linear_dict_start, sense='==', rhs=1, name='start_one_out')
    cost_func.linear_constraint(linear=linear_dict_target, sense='==', rhs=1, name='target_one_inc')

    return

In [ ]:
# execute QAOA on local simulator

# https://qiskit.org/documentation/stubs/qiskit.algorithms.QAOA.html?highlight=qaoa

# 'depth of circuit grows linearly with p times (at worst) the number of constraints'
# https://arxiv.org/pdf/1411.4028.pdf

optimizer = SPSA(maxiter=250)

qaoa = QAOA(reps=25,optimizer=optimizer,quantum_instance =
             QuantumInstance(backend=qiskit.Aer.get_backend('qasm_simulator')))
optimizer_qaoa = MinimumEigenOptimizer(qaoa)

results = []
inter_time = time.time()

for i in range(10):
    result_qaoa = optimizer_qaoa.solve(qiskit_cost_function)
    results.append(result_qaoa)
    print(result_qaoa)
    now = time.time()
    print(now - inter_time)
    inter_time = now


## Results

In [ ]:
# path cost: Wegkosten
# actual_opt_cost: Optimierungskosten. Schließt Wegkosten und Penalty-Terms ein. Ist immer positiv.
# path: qubit-ergebnis
result_df = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

for r in results:

    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]

    result_df = result_df.append({'actual_opt_cost': r.fval + 2 * penalty_value, 'path': path_string}, ignore_index=True)

print("QAOA:")
print(result_df.sort_values(by=['actual_opt_cost']))

In [ ]:
optimizer = CplexOptimizer() if CplexOptimizer.is_cplex_installed() else None

results_classic = []

for i in range(2):
    result = optimizer.solve(qiskit_cost_function)
    results_classic.append(result)
results_classic

In [ ]:

backend = qiskit.Aer.get_backend('qasm_simulator')

optimizer_type = 'SPSA' # JRL was COBYLA
maxiter = 500
if optimizer_type == 'COBYLA':
    optimizer = COBYLA(maxiter=maxiter) # JRL: was 500
elif optimizer_type == 'L_BFGS_B':
    optimizer = L_BFGS_B(maxfun=500)
elif optimizer_type == 'SPSA':
    optimizer = SPSA(maxiter=maxiter)
elif optimizer_type == 'SLSQP':
    optimizer = SLSQP(maxiter=maxiter)

#reps = 3 # JRL: parameter p in QAOA 

In [ ]:
# JRL adding VQE
from qiskit.circuit.library import TwoLocal
ry = TwoLocal(nr_qubits, 'ry', 'cz', reps=15, entanglement='linear')
vqe = VQE(ry, optimizer=optimizer, quantum_instance=QuantumInstance(backend=backend))

optimizer_vqe = MinimumEigenOptimizer(vqe)

results_vqe = []

#for i in range(10):
for i in range(10): # JRL
    print("i ",i) # JRL
    result_vqe = optimizer_vqe.solve(qiskit_cost_function)
    results_vqe.append(result_vqe)
    print("result_vqe ", result_vqe) # JRL


In [ ]:
result_vqe_df = pd.DataFrame(columns = ['actual_opt_cost', 'path'])

for r in results_vqe:
    
    path_string = str(r.x).replace(' ', '').replace('.', '')[1:-1]

    result_vqe_df = result_vqe_df.append({'actual_opt_cost': r.fval + 6 * penalty_value, 'path': path_string}, ignore_index=True)

print("VQE:")
print(result_vqe_df.sort_values(by=['actual_opt_cost']))

In [ ]:
# Parameters
print("TestP: ", penalty_value)
print("optimizer_type:", optimizer_type)
print("maxiter: ", maxiter)

In [ ]:
#result_df = pd.DataFrame(columns = ['optimization_cost', 'path'])
#for r in results:
#    result_string = str(r.x).replace(' ', '').replace('.', '').replace('-', '')[1:-1]
#    result_df = result_df.append({'optimization_cost': r.fval, 'path': result_string}, ignore_index=True)
#result_df

In [ ]:
result_classic_df = pd.DataFrame(columns = ['optimization_cost', 'path'])
for r in results_classic:
    result_string = str(r.x).replace(' ', '').replace('.', '').replace('-', '')[1:-1]
    result_classic_df = result_classic_df.append({'optimization_cost': r.fval + 4 * penalty_value, 'path': result_string}, ignore_index=True)
result_classic_df